## Notes 
### Différents type de régimes : 
Foodmap : Favorise certains aliments contenant des glucides capable de fermenter dans l'intestin. 
Méditerranéen : Favorise l'équilibre alimentaire, pour privilégier la variété des aliments. 
Dash (Dietary Approaches to Stopping Hypertension) : Permet de combattre l'hypertension artérielle en modifiant les habitudes alimentaires.
À indice glycémique bas : Régule le taux de sucre dans le sang en fonction des aliments et deux leur indice glycémique (IG).
Végétarien/Végétalisme : Exclut toutes chaires animales / Exclut la consommation de tous produits d'origine animale.
Paléolithique : Favorise la même alimentation que nos ancêtres, avant l'ère industrielle. 
Cétogène : Suppresion quasi complète des sucres et amidons(glucides).

### Menus en fonction des régimes : 
Paléolithique : https://www.passeportsante.net/fr/Nutrition/Regimes/Fiche.aspx?doc=paleolithique_regime

## Étapes d'avancements 
Étape 1 : Nettoyage des données<br>
Étape 2 : Importer les données dans une base de donnée <br>
Étape 3 : Création de menu, et de profil d'utilisateur (catégorie)<br>
Étape 4 : Importer ces nouvelles données dans la base de donnée <br>
Étape 5 : Mettre en corrélation les aliments avec les menus & utilisateur <br>

In [31]:
# Import nécessaires des packages 
import pandas as pd 

In [30]:
# Création d'un tableau en lisant le .csv fournit par openfoodfacts, en précisant le séparateur et le nombre de lignes car trop volumineux 
df = pd.read_csv("../data/en.openfoodfacts.org.products.csv", sep="\t", nrows=10000)

# On récupère les colonnes dans un fichier .csv pour pouvoir les consulter en entier
# df.columns.to_series().to_csv("colonnes.csv")

df 


C:\Users\Baptiste\AppData\Local\Temp\ipykernel_7492\1873976850.py:2: DtypeWarning: Columns (72) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/en.openfoodfacts.org.products.csv", sep="\t", nrows=10000)
